In [ ]:
import torch
import os
import cv2
import grayscale_moments
import resnet
import hog
import extract_features

In [ ]:
data1_path = '../data/Part1/Part1'
data2_path = '../data/Part2/Part2'

#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
#model.eval()

In [ ]:
from transformers import ViTConfig, ViTModel, ViTImageProcessor

# 1. Load the ViT config from your local folder
vision_config = ViTConfig.from_pretrained(
    "/workspaces/dbm25/data/vit_b16_224-mluke/vision_encoder"
)

# 2. Load the model weights into a ViTModel
model = ViTModel.from_pretrained(
    "/workspaces/dbm25/data/vit_b16_224-mluke/vision_encoder",
    config=vision_config,
    # map_location="cpu",  # uncomment if you need CPU-only
)

# 3. Set to eval mode if you’re doing inference
model.eval()

# 4. Prepare your images for ViT
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
# If you have PIL images in a list called `images`:
inputs = processor(images=rgb, return_tensors="pt")

outputs   = model(**inputs)
# Print the keys of the output and their shapes
for key, value in outputs.items():
    print(f"{key}: {value.shape}")

In [16]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
extracted_features = []


# For each image in data1_path extract the features
for root, dirs, files in os.walk(data2_path):
    for image in files:
        if not image.endswith('.jpg'):
            continue
        # Class can be glioma, menin, tumor. Get the class from the filename
        class_name = image.split('_')[0]+'_' + image.split('_')[1]
        image_path = os.path.join(f'{class_name}/', image)
        # Get the full path to the image
        image_path = os.path.abspath(os.path.join(data2_path, image_path))
        print(f'Image Path {image_path}')
        features_dict = extract_features.extract_features(image_path, model)
        
        # Append the features to the list
        extracted_features.append(features_dict)


Image Path /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1348.jpg
Processing /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1348.jpg
Image Path /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1360.jpg
Processing /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1360.jpg
Image Path /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1406.jpg
Processing /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1406.jpg
Image Path /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1412.jpg
Processing /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1412.jpg
Image Path /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1374.jpg
Processing /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1374.jpg
Image Path /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1638.jpg
Processing /workspaces/dbm25/data/Part2/Part2/brain_menin/brain_menin_1638.jpg
Image Path /workspaces/dbm25/data/Part2/Part2/brain_

In [20]:
# Save the extracted features into a pt file
torch.save(extracted_features, '../data/extracted_features_part_2.pt')

In [ ]:
# Load the extracted features from the pt file
extracted_features_loaded = torch.load('../data/extracted_features.pt')

In [ ]:
part2_test_features = extract_features.extract_features(f'{data2_path}/brain_glioma/brain_glioma_1962.jpg', model)
print(part2_test_features)
avgpool_test = part2_test_features['layer3']

In [ ]:
first_elem = extracted_features[0]
# Print the shape of each key in the dictionary
for key in first_elem.keys():
    # Check if it's a tensor
    if isinstance(first_elem[key], torch.Tensor):
        print(f'{key}: {first_elem[key].shape}')

In [ ]:
first_elem_loaded = extracted_features_loaded[0]
# Print the shape of each key in the dictionary
for key in first_elem_loaded.keys():
    # Check if it's a tensor
    if isinstance(first_elem_loaded[key], torch.Tensor):
        print(f'{key}: {first_elem_loaded[key].shape}')